In [1]:
import sys
import os

# Path to the folder containing ipyformkit (one level up from examples/)
project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Project root added to sys.path: {project_root}")
# Try importing now
import ipyformkit as ifk

Project root added to sys.path: c:\Users\Richard\Desktop\Notebooks\IpyFormKit


In [2]:
import ipywidgets as widgets
from IPython.display import display, HTML

test = widgets.Button(description='Click me!')
test.add_class('text')
display(test)

Button(description='Click me!', style=ButtonStyle(), _dom_classes=('text',))

In [8]:
from decimal import Decimal

#=====================================================================
def count_decimal_places(value: float) -> int:
    # Convert via string to preserve user input formatting
    d = Decimal(str(value)).normalize()
    # Negative exponent means decimal places
    return abs(d.as_tuple().exponent)
    
#=====================================================================
def create_widget(key, value):
    label = widgets.Label(value=key)
    label.add_class('ifk-widget-label')
    
    box = widgets.VBox([label])
    box.add_class('ifk-widget-box')
    
    if isinstance(value, bool):
        wid = widgets.Checkbox(value=value, description=key, indent=False)
        box.add_class('ifk-widget-Checkbox')
        label.value = ''
    elif isinstance(value, int):
        wid = widgets.IntText(value=value)
        box.add_class('ifk-widget-IntText')
    elif isinstance(value, float):
        step = 10**(-count_decimal_places(value))
        wid = widgets.FloatText(value=value, step=step)
        box.add_class('ifk-widget-FloatText')
    elif isinstance(value, str):
        if os.sep in value:
            wid = ifk.FileAutocomplete(placeholder=value)
            box.add_class('ifk-widget-FileAutocomplete')
        else:
            wid = widgets.Text(placeholder=value)
            box.add_class('ifk-widget-Text')
    elif isinstance(value, list):
        if value:
            wid = widgets.Dropdown(options=value, value=value[0])
            box.add_class('ifk-widget-Dropdown')
        else:
            wid = widgets.Label(value="(Empty list - no options)")
            box.add_class('ifk-widget-Label')
    else:
        wid = widgets.Label(value=f"Unsupported type: {type(value).__name__}")
        box.add_class('ifk-widget-Label')
    
    wid.add_class('ifk-widget-input')
    box.children = list(box.children) + [wid,]
    box.label = label
    box.wid = wid
    return box

#=====================================================================
def dict_to_form(input_dict, title=None, collapsed=None, nested=False):
    widgets_list = []
    widgets_dict = {}

    if collapsed is None and title is not None:
        title_widget = widgets.Label(value=title)
        title_widget.add_class('ifk-form-title')
        widgets_list.append(title_widget)

    for key, value in input_dict.items():
        hbox_items = []
        
        # Case: nested tuple group
        if isinstance(key, tuple) and isinstance(value, tuple):
            for sub_key, sub_val in zip(key, value):
                wid = create_widget(sub_key, sub_val)
                widgets_dict[sub_key] = wid
                hbox_items.append(wid)

        elif isinstance(value, dict):
            # Case: nested dictionary group
            sub_vbox, sub_widgets_dict = dict_to_form(value, title=key, collapsed=True, nested=True)
            widgets_dict.update(sub_widgets_dict)
            hbox_items.append(sub_vbox)
            
        else:
            wid = create_widget(key, value)
            widgets_dict[key] = wid
            hbox_items.append(wid)
            
        hbox = widgets.HBox(hbox_items)
        hbox.add_class('ifk-form-hbox')
        widgets_list.append(hbox)
        
    if collapsed is not None:
        vbox = ifk.CollapsibleVBox(widgets_list, title=title, collapsed=collapsed)
        vbox.toggle_button.add_class('ifk-form-toggle-button')
        vbox.label.add_class('ifk-widget-label')
        vbox.layout.width = '100%'
    else:
        vbox = widgets.VBox(widgets_list)

    if not nested: vbox.add_class('ifk-form')

    return vbox, widgets_dict

#=====================================================================
class IpyForm(object):
    def __init__(self, input_dict, title=None, collapsed=None, max_width=600,
                 mandatory=None):
        
        self.input_dict = input_dict
        self.title = title
        self.collapsed = collapsed
        self.mandatory = mandatory

        self.vbox, self.widgets_dict = dict_to_form(input_dict, title=title, collapsed=collapsed)
        self.vbox.layout.max_width = f'{max_width}px'

        if isinstance(mandatory, dict):
            for key, val in mandatory.items():
                if val:
                    wid = self.widgets_dict[key]
                    wid.label.value = f"{wid.label.value} *"

    def display(self):
        display(self.vbox)

    def get_values(self):
        out = {key: wid.wid.value for key, wid in self.widgets_dict.items()}
        if self.mandatory is not None:
            for key, val in self.mandatory.items():
                if val and out[key] == '':
                    raise ValueError(f"Mandatory field '{key}' is empty.")
        return out
    

In [9]:
import ipyformkit as ifk

test = {'first name':'John',
        'last name':'Doe',
        ('age', 'gender'):(42, ['male', 'female', 'non-binary']),
        'height':1.82,
        'date of birth':'01.01.1980',
        ('input 1', 'input 2', 'input 3', 'input 4', 'input 5'):(1,2,3,4,5),
        ('street', 'house'):('Baker Street', 42),
        'additional info':{'hobby':'reading',
                      'favorite color':'blue',
                      'favorite food':'pizza',
                      'favorite movie':'Inception',
                      'favorite book':'1984'},
       }

mandatory = {'first name': True, 'last name': True, 'age':True}

form = IpyForm(test, title='Test Form', mandatory=mandatory)
form.display()

In [7]:
form.get_values()

ValueError: Mandatory field 'first name' is empty.